In [ ]:
!pip install pytorch-metric-learning -q


In [2]:
# Fix numpy import error first
import sys
import subprocess
import os

# Fix numpy environment issues
os.environ['OPENBLAS_NUM_THREADS'] = '1'

try:
    import numpy as np
    print(f"✅ NumPy version: {np.__version__}")
except ImportError:
    print("❌ NumPy import failed. Reinstalling...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "numpy==1.21.6"])
    import numpy as np
    print(f"✅ NumPy reinstalled: {np.__version__}")

✅ NumPy version: 2.0.2


In [5]:
import sys
import subprocess
import os
import warnings
warnings.filterwarnings('ignore')

# Set environment variables to fix numpy issues
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

print("🔧 Fixing numpy import issues...")

# Force reinstall numpy with specific version
try:
    # First try importing
    import numpy as np
    print(f"✅ NumPy version: {np.__version__}")
    
    # Test numpy functionality
    test_array = np.array([1, 2, 3])
    test_result = np.mean(test_array)
    print(f"✅ NumPy functionality test: {test_result}")
    
except (ImportError, AttributeError) as e:
    print(f"❌ NumPy error: {e}")
    print("🔄 Reinstalling numpy...")
    
    # Uninstall and reinstall numpy
    subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "numpy", "-y"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy==1.21.6", "--no-cache-dir"])
    
    # Import again
    import numpy as np
    print(f"✅ NumPy reinstalled: {np.__version__}")

print("✅ NumPy setup completed")

🔧 Fixing numpy import issues...
✅ NumPy version: 2.0.2
✅ NumPy functionality test: 2.0
✅ NumPy setup completed


In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
from PIL import Image
def TEST(folder_path, model, transforms, key):

    label_org = []
    dir_org = []
    label_test = []
    dir_test_path = []
    dir_org_path = []
    REFER_DICT = {}

    # Put the model in evaluation mode
    model.eval()

    # Disable gradient calculation
    with torch.no_grad():
        # Iterate over each subfolder in the folder_path
        for label_index, subfolder_name in enumerate(os.listdir(folder_path)):
            REFER_DICT[label_index] = subfolder_name
            subfolder_path = os.path.join(folder_path, subfolder_name)
            image_files = os.listdir(subfolder_path)
            for image_index, image_file in enumerate(image_files):

                image_path = os.path.join(subfolder_path, image_file)

                if key in image_path:

                    image = Image.open(image_path).convert('RGB')
                    image = transforms(image=np.array(image))["image"]
                    # Extract the embedding for the first image in the folder
                    embedding = model(image.unsqueeze(0).to("cuda"))
                    dir_org.append(embedding)
                    label_org.append(label_index)
                    dir_org_path.append(image_path)
                else:
                    # Store the path and label for other images
                    dir_test_path.append(image_path)
                    label_test.append(label_index)

        predict_label = []
        Max_sim = []

        if not dir_org:  # Nếu không có ảnh reference
            print(f"Warning: No reference images found with key '{key}'")
            return 0.0

        # Iterate over test images
        for test_image_path in dir_test_path:

            test_image = Image.open(test_image_path).convert('RGB')
            test_image = transforms(image=np.array(test_image))["image"]

            # Extract the embedding for the test image
            test_embedding = model(test_image.unsqueeze(0).to("cuda"))
            similarities = []

            # Calculate cosine similarity with each original embedding
            for org_embedding in dir_org:

                cosine_sim = cosine_similarity(org_embedding.cpu().detach().numpy(), test_embedding.cpu().detach().numpy())
                similarities.append(cosine_sim[0][0])

            if similarities:
                Max_sim.append(max(similarities))
                max_similarity_index = np.argmax(similarities)
                predict_label.append(label_org[max_similarity_index])
            else:
                print(f"Warning: No similarities calculated for {test_image_path}")
                predict_label.append(-1)  # Hoặc một giá trị mặc định
        if not predict_label or not label_test:
            print("Warning: No predictions or labels to evaluate")
            return 0.0
                
        accuracy = accuracy_score(predict_label, label_test)
        print(f'----Key: {key}')
        print(f'----Number of test images: {len(label_test)}')
        print(f'----Number of reference images: {len(label_org)}')
        print(f'----Accuracy: {accuracy:.4f}')
        print()

        return accuracy

preprocess = A.Compose([
        A.Resize(224, 224),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\DELL\miniconda3\envs\faceid\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\DELL\miniconda3\envs\faceid\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\DELL\miniconda3\envs\faceid\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\DELL\miniconda3\envs\faceid\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
#load path_model = "SupConLoss_BBModel2\Model2\SupConLoss_BBVGG16.pth"
model = models.vgg16(pretrained=False)
model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 256)
)
model = model.to("cuda")
model_path = "SupConLoss_BBModel2\Model2\SupConLoss_BBVGG16.pth"  # Ví dụ path
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location="cuda")
    model.load_state_dict(checkpoint['model_state_dict'])
else:
    print(f"Model path {model_path} does not exist. Please check the path.")

# Execute analysis
test_path = "logo_verify_test"  # Thay đổi path theo dữ liệu của bạn
if os.path.exists(test_path):
    results = TEST(test_path, model, preprocess, key="000000")
else:
    print(f"Test path {test_path} does not exist. Please check the path.")
if results is not None:
    print(f"Final accuracy: {results:.4f}")
else:
    print("No results to display.")
